In [4]:
# Set up problem using TrajectoryOptimization.jl and RobotZoo.jl
using TrajectoryOptimization
using Altro
import RobotZoo.Cartpole
using StaticArrays, LinearAlgebra
using RobotDynamics


# Use the Cartpole model from RobotZoo
model = Cartpole()
print(typeof(dynamics(model, [0,0,0,0], [0,0])))
n,m = size(model)

# Define model discretization
N = 101
tf = 5.
dt = tf/(N-1)

# Define initial and final conditions
x0 = @SVector zeros(n)
xf = @SVector [0, pi, 0, 0]  # i.e. swing up

# Set up
Q = 1.0e-2*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 1.0e-1*Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N)

# Add constraints
conSet = ConstraintList(n,m,N)
u_bnd = 3.0
bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
goal = GoalConstraint(xf)
add_constraint!(conSet, bnd, 1:N-1)
add_constraint!(conSet, goal, N)

# Initialization
u0 = @SVector fill(0.01,m)
U0 = [u0 for k = 1:N-1]

# Define problem
prob = Problem(model, obj, xf, tf, x0=x0, constraints=conSet)
initial_controls!(prob, U0)

SVector{4, Float64}Problem{RK3, Float64}(Cartpole{Float64}(1.0, 0.2, 0.5, 9.81), Objective, ConstraintList(4, 1, TrajectoryOptimization.AbstractConstraint[BoundConstraint{2, 5, Float64}(4, 1, [Inf, Inf, Inf, Inf, 3.0], [-Inf, -Inf, -Inf, -Inf, -3.0], [9], [10], [5, 10]), GoalConstraint{4, Float64}(4, [0.0, 3.141592653589793, 0.0, 0.0], [1, 2, 3, 4])], UnitRange{Int64}[1:100, 101:101], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4]), [0.0, 0.0, 0.0, 0.0], [0.0, 3.141592653589793, 0.0, 0.0], KnotPoint{Float64, 4, 1, 5}[KnotPoint{Float64, 4, 1, 5}([NaN, NaN, NaN, NaN, 0.01], [1, 2, 3, 4], [5], 0.05, 0.0), KnotPoint{Float64, 4, 1, 5}([NaN, NaN, NaN, NaN, 0.01], [1, 2, 3, 4], [5], 0.05, 0.05), KnotPoint{Float64, 4, 1, 5}([NaN, NaN, NaN, N

In [9]:
# Solve with ALTRO
opts = SolverOptions(
    cost_tolerance_intermediate=1e-2,
    penalty_scaling=10.,
    penalty_initial=1.0
)
altro = ALTROSolver(prob, opts)
solve!(altro)

# Get some info on the solve
max_violation(altro)  # 3.42e-9
cost(altro)           # 1.55
iterations(altro)     # 40

# Extract the solution
X = states(altro)
U = controls(altro)

# Extract the solver statistics
stats = Altro.stats(altro)  # alternatively, solver.stats
stats.iterations             # 40, equivalent to iterations(solver)
stats.iterations_outer       # 4 (number of Augmented Lagrangian iterations)
stats.iterations_pn          # 1 (number of projected newton iterations)
stats.cost[end]              # terminal cost
stats.c_max[end]             # terminal constraint satisfaction
stats.gradient[end]          # terminal gradient of the Lagrangian
dstats = Dict(stats)         # get the per-iteration stats as a dictionary (can be converted to DataFrame)


SOLVE COMPLETED
 solved using the ALTRO Solver,
 part of the Altro.jl package developed by the REx Lab at Stanford and Carnegie Mellon Universities

  Solve Statistics
    Total Iterations: 19
    Solve Time: 8.943602 (ms)

  Covergence
    Terminal Cost: 1.5055212913511913
    Terminal dJ: -1.7491631070939175
    Terminal gradient: 0.0011467833210839448
    Terminal constraint violation: 3.7731547131159004e-7
    Solve Status: SOLVE_SUCCEEDED


Dict{Symbol, Vector{T} where T} with 8 entries:
  :gradient        => [0.0383027, 0.0124697, 0.0209221, 0.0248707, 0.049519, 0.…
  :cost            => [1.50776, 1.50765, 1.50702, 1.50865, 1.52418, 1.68006, 3.…
  :iteration_pn    => Bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0…
  :iteration       => [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 1…
  :c_max           => [0.0225664, 0.0225664, 0.00688533, 0.00380926, 0.00377652…
  :penalty_max     => [1.0, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1.0e6,…
  :iteration_outer => [1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8]
  :dJ              => [0.00616176, 0.0104677, 0.000627835, 0.00112107, 0.000121…